# Splitting Data

Here, we utilize the feature-selected profiles generated in the preceding module notebook [here](../0.feature_selection/0.feature_selection.ipynb), focusing on dividing the data into training, testing, and holdout sets for machine learning training.


In [1]:
import sys
import json
import pathlib
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

sys.path.append("../../")  # noqa
from src.utils import split_meta_and_features, get_injury_treatment_info, load_json_file  # noqa

# ignoring warnings
warnings.catch_warnings(action="ignore")

catch_warnings()

## Parameters

Below are the parameters defined that are used in this notebook


In [2]:
# setting seed constants
seed = 0
np.random.seed(seed)

# directory to get all the inputs for this notebook
data_dir = pathlib.Path("../../data").resolve(strict=True)
results_dir = pathlib.Path("../../results").resolve(strict=True)
fs_dir = (results_dir / "0.feature_selection").resolve(strict=True)

# directory to store all the output of this notebook
data_split_dir = (results_dir / "1.data_splits").resolve()
data_split_dir.mkdir(exist_ok=True)

In [3]:
# data paths
fs_profile_path = (fs_dir / "cell_injury_profile_fs.csv.gz").resolve(strict=True)

# load in feature space file
fs_feature_space = load_json_file(
    fpath=(fs_dir / "cell_injury_shared_feature_space.json").resolve(strict=True)
)
fs_feats = fs_feature_space["features"]

# load data
fs_profile_df = pd.read_csv(fs_profile_path)

# splitting meta and feature column names
fs_meta, _ = split_meta_and_features(fs_profile_df)

# display
print("fs profile with control: ", fs_profile_df.shape)
fs_profile_df.head()

fs profile with control:  (16703, 254)


,injury_code,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,...,Nuclei_RadialDistribution_RadialCV_ER_3of4,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_RadialDistribution_RadialCV_RNA_2of4,Cells_AreaShape_Zernike_7_7,Cells_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MeanIntensityEdge_AGP,Cells_RadialDistribution_RadialCV_DNA_2of4,Nuclei_Intensity_IntegratedIntensity_RNA,Nuclei_Intensity_MassDisplacement_RNA
0,0,Control,BR00110363,B2,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.024875,0.021860,-0.021390,-0.010154,-0.012403,-0.084749,0.164219,0.008127,0.145635,-0.049983
1,0,Control,BR00110363,B3,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,-0.000281,-0.019313,-0.001698,-0.092282,0.046258,-0.026739,0.041108,-0.078602,0.112177,-0.017450
2,0,Control,BR00110363,B4,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.000791,0.010803,0.017969,-0.069081,0.011242,-0.016767,-0.036748,0.001861,0.196431,-0.033089
3,0,Control,BR00110363,B5,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.015091,0.002641,-0.028728,-0.044338,-0.000466,-0.029708,-0.010994,-0.021929,0.023909,-0.027086
4,0,Control,BR00110363,B6,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,-0.026855,-0.019703,-0.009698,-0.005962,0.034776,0.103990,0.068561,0.008069,0.051912,0.001262


## Exploring the data set

Below is a exploration of the selected features dataset. The aim is to identify treatments, extract metadata, and gain a understanding of the experiment's design.


Below demonstrates the amount of wells does each treatment have.


In [4]:
# displaying the amount of wells per treatments
well_treatments_counts_df = (
    fs_profile_df["Compound Name"].value_counts().to_frame().reset_index()
)

well_treatments_counts_df

,Compound Name,count
0,DMSO,9855
1,Wortmannin,600
2,Colchicine,512
3,Nocodazole,504
4,Radicicol,504
...,...,...
139,Melphalan,24
140,Carmustine,24
141,Thio-TEPA,24
142,Chlorambucil,24


Below we show the amount of wells does a specific cell celluar injury has


In [5]:
# Displaying how many how wells does each cell injury have
cell_injury_well_counts = (
    fs_profile_df["injury_type"].value_counts().to_frame().reset_index()
)
cell_injury_well_counts

,injury_type,count
0,Control,9855
1,Cytoskeletal,1472
2,Miscellaneous,1304
3,Kinase,1104
4,Genotoxin,944
5,Hsp90,552
6,Redox,312
7,Saponin,288
8,HDAC,168
9,Proteasome,144


Next we wanted to extract some metadata regarding how many compound and wells are treated with a given compounds

This will be saved in the `results/0.data_splits` directory


In [6]:
# get summary information and save it
injury_before_holdout_info_df = get_injury_treatment_info(
    profile=fs_profile_df, groupby_key="injury_type"
).reset_index(drop=True)

# display
print("Shape:", injury_before_holdout_info_df.shape)
injury_before_holdout_info_df

Shape: (15, 5)


,injury_type,injury_code,n_wells,n_compounds,compound_list
0,Control,0,9855,1,[DMSO]
1,Cytoskeletal,1,1472,15,"[Nocodazole, Colchicine, Paclitaxel, Vinblasti..."
2,Miscellaneous,5,1304,39,"[L-Buthionine-(S,R)-sulfoximine, CDDO Im, Cino..."
3,Kinase,3,1104,13,"[Wortmannin, Staurosporine, PI-103, BEZ-235, A..."
4,Genotoxin,4,944,22,"[Camptothecin, CX-5461, Doxorubicin, Cladribin..."
5,Hsp90,2,552,3,"[Radicicol, Geldanamycin, 17-AAG]"
6,Redox,6,312,12,"[Menadione, PKF118-310, 4-Amino-1-naphthol (HC..."
7,Saponin,10,288,11,"[Digitonin, Saikosaponin A, Polygalasaponin F,..."
8,HDAC,7,168,5,"[AR-42, SAHA, ITF 2357, Panobinostat, Apicidin]"
9,Mitochondria,11,144,4,"[Antimycin A, CCCP, Rotenone, Oligomycin A]"


Next, we construct the profile metadata. This provides a structured overview of how the treatments assicoated with injuries were applied, detailing the treatments administered to each plate.

This will be saved in the `results/0.data_splits` directory


In [7]:
injury_meta_dict = {}
for injury, df in fs_profile_df.groupby("injury_type"):
    # collecting treatment metadata
    plates = df["Plate"].unique().tolist()
    treatment_meta = {}
    treatment_meta["n_plates"] = len(plates)
    treatment_meta["n_wells"] = df.shape[0]
    treatment_meta["n_treatments"] = len(df["Compound Name"].unique())
    treatment_meta["associated_plates"] = plates

    # counting treatments
    treatment_counter = {}
    for treatment, df2 in df.groupby("Compound Name"):
        if treatment is np.nan:
            continue
        n_treatments = df2.shape[0]
        treatment_counter[treatment] = n_treatments

    # storing treatment counts
    treatment_meta["treatments"] = treatment_counter
    injury_meta_dict[injury] = treatment_meta

# save dictionary into a json file
with open(data_split_dir / "injury_metadata.json", mode="w") as stream:
    json.dump(injury_meta_dict, stream)

Here we build a plate metadata information where we look at the type of treatments and amount of wells with the treatment that are present in the dataset

This will be saved in `results/0.data_splits`


In [8]:
plate_meta = {}
for plate_id, df in fs_profile_df.groupby("Plate"):
    unique_compounds = list(df["Compound Name"].unique())
    n_treatments = len(unique_compounds)

    # counting treatments
    treatment_counter = {}
    for treatment, df2 in df.groupby("Compound Name"):
        n_treatments = df2.shape[0]
        treatment_counter[treatment] = n_treatments

    plate_meta[plate_id] = treatment_counter

# save dictionary into a json file
with open(data_split_dir / "cell_injury_plate_info.json", mode="w") as stream:
    json.dump(plate_meta, stream)

## Data Splitting

---


### Holdout Dataset

Here we collected out holdout dataset. The holdout dataset is a subset of the dataset that is not used during model training or tuning. Instead, it is reserved solely for evaluating the model's performance after it has been trained.

In this notebook, we will include three different types of held-out datasets before proceeding with our machine learning training and evaluation.

- Plate hold out
- treatment hold out
- well hold out

Each of these held outdata will be stored in the `results/1.data_splits` directory


### Plate Holdout

Plates are randomly selected based on their Plate ID and save them as our `plate_holdout` data.


In [9]:
# plate
n_plates = 10

# setting random seed globally
np.random.seed(seed)

# selecting plates randomly from a list
selected_plates = (
    np.random.choice(fs_profile_df["Plate"].unique().tolist(), (n_plates, 1))
    .flatten()
    .tolist()
)
plate_holdout_df = fs_profile_df.loc[fs_profile_df["Plate"].isin(selected_plates)]

# take the indices of the held out data frame and use it to drop those samples from
# the main dataset. And then check if those indices are dropped
plate_idx_to_drop = plate_holdout_df.index.tolist()
fs_profile_df = fs_profile_df.drop(plate_idx_to_drop)
assert all(
    [
        True if num not in fs_profile_df.index.tolist() else False
        for num in plate_idx_to_drop
    ]
), "index to be dropped found in the main dataframe"

# saving the holdout data
plate_holdout_df.to_csv(
    data_split_dir / "plate_holdout.csv.gz", index=False, compression="gzip"
)

# display
print("plate holdout shape:", plate_holdout_df.shape)
plate_holdout_df.head()

plate holdout shape: (1949, 254)


,injury_code,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,...,Nuclei_RadialDistribution_RadialCV_ER_3of4,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_RadialDistribution_RadialCV_RNA_2of4,Cells_AreaShape_Zernike_7_7,Cells_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MeanIntensityEdge_AGP,Cells_RadialDistribution_RadialCV_DNA_2of4,Nuclei_Intensity_IntegratedIntensity_RNA,Nuclei_Intensity_MassDisplacement_RNA
1044,0,Control,BR00110368,B2,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.010870,0.016349,0.021725,-0.022160,-0.000081,0.031287,0.124336,-0.015969,0.252456,0.015518
1045,0,Control,BR00110368,B3,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.077955,0.024034,-0.033816,-0.095424,0.036521,-0.090489,0.053509,-0.060504,0.285266,0.022204
1046,0,Control,BR00110368,B4,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.036134,0.020462,-0.011504,-0.041865,0.009526,-0.066667,0.064256,-0.032779,0.172206,-0.008920
1047,0,Control,BR00110368,B5,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.002204,0.006864,-0.019585,-0.101737,0.014732,-0.029291,0.035350,-0.123300,0.013471,0.005946
1048,0,Control,BR00110368,B6,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,-0.015167,0.012200,0.017024,-0.097446,0.015704,0.097898,0.085943,-0.029772,0.128706,-0.022323


### Treatment holdout

To establish our treatment holdout, we first need to find the number of treatments and wells associated with a specific cell injury, considering the removal of randomly selected plates from the previous step.

To determine which cell injuries should be considered for a single treatment holdout, we establish a threshold of 10 unique compounds. This means that a cell injury type must have at least 10 unique compounds to qualify for selection in the treatment holdout. Any cell injury types failing to meet this criterion will be disregarded.

Once the cell injuries are identified for treatment holdout, we select our holdout treatment by grouping each injury type and choosing the treatment with the fewest wells. This becomes our treatment holdout dataset.


In [10]:
injury_treatment_metadata = (
    fs_profile_df.groupby(["injury_type", "Compound Name"])
    .size()
    .reset_index(name="n_wells")
)
injury_treatment_metadata

,injury_type,Compound Name,n_wells
0,Control,DMSO,8783
1,Cytoskeletal,ARQ 621,12
2,Cytoskeletal,Citreoviridin,18
3,Cytoskeletal,Citrinin,18
4,Cytoskeletal,Colchicine,457
...,...,...,...
139,Tannin,Corilagin,18
140,Tannin,Gallotannin,24
141,Tannin,Punicalagin,18
142,mTOR,Rapamycin,42


In [11]:
# setting random seed
min_treatments_per_injury = 10

# Filter out the injury types for which we can select a complete treatment.
# We are using a threshold of 10. If an injury type is associated with fewer than 10 compounds,
# we do not conduct treatment holdout on those injury types.
accepted_injuries = []
for injury_type, df in injury_treatment_metadata.groupby("injury_type"):
    n_treatments = df.shape[0]
    if n_treatments >= min_treatments_per_injury:
        accepted_injuries.append(df)

accepted_injuries = pd.concat(accepted_injuries)

# Next, we select the treatment that will be held out within each injury type.
# We group treatments based on injury type and choose the treatment with the fewest wells
# as our holdout.
selected_treatments_to_holdout = []
for injury_type, df in accepted_injuries.groupby("injury_type"):
    held_treatment = df.min().iloc[1]
    selected_treatments_to_holdout.append([injury_type, held_treatment])

# convert to dataframe
selected_treatments_to_holdout = pd.DataFrame(
    selected_treatments_to_holdout, columns="injury_type held_treatment".split()
)

print("Below are the accepted cell injuries and treatments to be held out")
selected_treatments_to_holdout

Below are the accepted cell injuries and treatments to be held out


,injury_type,held_treatment
0,Cytoskeletal,ARQ 621
1,Genotoxin,Aphidicolin
2,Kinase,AZD 1152-HQPA
3,Miscellaneous,Aloisine RP106
4,Redox,4-Amino-1-naphthol (HCl)
5,Saponin,Bacopasaponin C


In [12]:
# select all wells that have the treatments to be heldout
treatment_holdout_df = fs_profile_df.loc[
    fs_profile_df["Compound Name"].isin(
        selected_treatments_to_holdout["held_treatment"]
    )
]

# take the indices of the held out data frame and use it to drop those samples from
# the main dataset. And then check if those indices are dropped
treatment_idx_to_drop = treatment_holdout_df.index.tolist()
fs_profile_df = fs_profile_df.drop(treatment_idx_to_drop)
assert all(
    [
        True if num not in fs_profile_df.index.tolist() else False
        for num in treatment_idx_to_drop
    ]
), "index to be dropped found in the main dataframe"
# saving the holdout data
treatment_holdout_df.to_csv(
    data_split_dir / "treatment_holdout.csv.gz", index=False, compression="gzip"
)

# display
print("Treatment holdout shape:", treatment_holdout_df.shape)
treatment_holdout_df.head()

Treatment holdout shape: (126, 254)


,injury_code,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,...,Nuclei_RadialDistribution_RadialCV_ER_3of4,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_RadialDistribution_RadialCV_RNA_2of4,Cells_AreaShape_Zernike_7_7,Cells_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MeanIntensityEdge_AGP,Cells_RadialDistribution_RadialCV_DNA_2of4,Nuclei_Intensity_IntegratedIntensity_RNA,Nuclei_Intensity_MassDisplacement_RNA
10865,1,Cytoskeletal,BR00114093,G17,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.939894,-0.355823,-0.108929,1.107614,-0.035961,-0.848475,-1.786879,0.558574,-0.361379,0.217145
10866,1,Cytoskeletal,BR00114093,G18,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.663508,-0.216140,-0.008218,0.627211,-0.011467,0.453441,0.037376,0.080870,0.393919,0.261334
10867,1,Cytoskeletal,BR00114093,G19,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.317817,-0.058756,-0.020949,0.540544,0.006174,-0.082120,-0.028223,-0.002127,-0.060136,0.196123
10868,1,Cytoskeletal,BR00114093,G20,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.131182,-0.040143,0.004868,0.131435,0.004184,0.160256,-0.014342,0.095674,-0.110048,0.112280
10869,1,Cytoskeletal,BR00114093,G21,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.081793,-0.001937,0.001392,0.045974,0.014768,0.205445,-0.088853,0.075584,-0.098539,0.098476


### Well holdout

To generate the well hold out data, each plate was iterated and random wells were selected. However, an additional step was conducting which was to separate the control wells and the treated wells, due to the large label imbalance with the controls. Therefore, 5 wells were randomly selected and 10 wells were randomly selected from each individual plate


In [13]:
# parameters
n_controls = 5
n_samples = 10

# setting random seed globally
np.random.seed(seed)

# collecting randomly select wells based on treatment
wells_heldout_df = []
for treatment, df in fs_profile_df.groupby("Plate", as_index=False):
    # separate control wells and rest of all wells since there is a huge label imbalance
    # selected 5 control wells and 10 random wells from the plate
    df_control = df.loc[df["Compound Name"] == "DMSO"].sample(
        n=n_controls, random_state=seed
    )
    df_treated = df.loc[df["Compound Name"] != "DMSO"].sample(
        n=n_samples, random_state=seed
    )

    # concatenate those together
    well_heldout = pd.concat([df_control, df_treated])

    wells_heldout_df.append(well_heldout)

# genearte treatment holdout dataframe
wells_heldout_df = pd.concat(wells_heldout_df)

# take the indices of the held out data frame and use it to drop those samples from
# the main dataset. And then check if those indices are dropped
wells_idx_to_drop = wells_heldout_df.index.tolist()
fs_profile_df = fs_profile_df.drop(wells_idx_to_drop)
assert all(
    [
        True if num not in fs_profile_df.index.tolist() else False
        for num in treatment_idx_to_drop
    ]
), "index to be dropped found in the main dataframe"

# saving the holdout data
wells_heldout_df.to_csv(
    data_split_dir / "wells_holdout.csv.gz", index=False, compression="gzip"
)

# display
print("Wells holdout shape:", wells_heldout_df.shape)
wells_heldout_df.head()

Wells holdout shape: (1125, 254)


,injury_code,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,...,Nuclei_RadialDistribution_RadialCV_ER_3of4,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_RadialDistribution_RadialCV_RNA_2of4,Cells_AreaShape_Zernike_7_7,Cells_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MeanIntensityEdge_AGP,Cells_RadialDistribution_RadialCV_DNA_2of4,Nuclei_Intensity_IntegratedIntensity_RNA,Nuclei_Intensity_MassDisplacement_RNA
4994,0,Control,BR00109990,B12,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.112750,0.013628,0.014515,-0.057282,0.005057,-0.141857,-0.240376,-0.015658,-0.086936,0.071032
5058,0,Control,BR00109990,K10,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.080362,-0.023308,0.033779,0.137843,-0.003944,0.011395,0.155140,0.056526,0.111172,-0.001147
5050,0,Control,BR00109990,J18,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.014751,-0.028560,0.025125,0.057508,-0.010651,0.174274,0.016046,0.069656,0.143133,-0.014077
5035,0,Control,BR00109990,G9,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,-0.003606,-0.015946,0.018551,0.034239,0.049856,-0.047922,0.147716,0.002586,0.234429,-0.023309
4991,0,Control,BR00109990,B9,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,-0.047512,-0.010211,-0.000080,-0.037869,0.015637,-0.163193,-0.237637,-0.060755,-0.005586,-0.010067


## Saving training dataset


Once the data holdout has been generated, the next step is to save the training dataset that will serve as the basis for training the multi-class logistic regression model.


In [14]:
# get summary cell injury dataset treatment and well info after holdouts
injury_after_holdout_info_df = get_injury_treatment_info(
    profile=fs_profile_df, groupby_key="injury_type"
)

# display
print("shape:", injury_after_holdout_info_df.shape)
injury_after_holdout_info_df

shape: (15, 5)


,injury_type,injury_code,n_wells,n_compounds,compound_list
0,Control,0,8408,1,[DMSO]
1,Cytoskeletal,1,1102,14,"[Nocodazole, Colchicine, Paclitaxel, Vinblasti..."
7,Miscellaneous,5,1007,38,"[L-Buthionine-(S,R)-sulfoximine, CDDO Im, Cino..."
6,Kinase,3,750,12,"[Wortmannin, Staurosporine, PI-103, BEZ-235, S..."
3,Genotoxin,4,737,21,"[Camptothecin, CX-5461, Doxorubicin, Cladribin..."
5,Hsp90,2,418,3,"[Radicicol, Geldanamycin, 17-AAG]"
11,Redox,6,215,11,"[Menadione, PKF118-310, Dunnione, MGR2, SIN-1 ..."
12,Saponin,10,164,10,"[Digitonin, Saikosaponin A, Polygalasaponin F,..."
4,HDAC,7,138,5,"[AR-42, SAHA, ITF 2357, Panobinostat, Apicidin]"
10,Proteasome,9,117,4,"[Carfilzomib, Bortezomib, (S)-MG132, (R)-MG132]"


In [15]:
# shape of the update training and testing dataset after removing holdout
print("training shape after removing holdouts", fs_profile_df.shape)
fs_profile_df.head()

training shape after removing holdouts (13503, 254)


,injury_code,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,...,Nuclei_RadialDistribution_RadialCV_ER_3of4,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_RadialDistribution_RadialCV_RNA_2of4,Cells_AreaShape_Zernike_7_7,Cells_Intensity_IntegratedIntensity_DNA,Cytoplasm_Intensity_MeanIntensityEdge_AGP,Cells_RadialDistribution_RadialCV_DNA_2of4,Nuclei_Intensity_IntegratedIntensity_RNA,Nuclei_Intensity_MassDisplacement_RNA
0,0,Control,BR00110363,B2,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.024875,0.021860,-0.021390,-0.010154,-0.012403,-0.084749,0.164219,0.008127,0.145635,-0.049983
1,0,Control,BR00110363,B3,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,-0.000281,-0.019313,-0.001698,-0.092282,0.046258,-0.026739,0.041108,-0.078602,0.112177,-0.017450
2,0,Control,BR00110363,B4,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.000791,0.010803,0.017969,-0.069081,0.011242,-0.016767,-0.036748,0.001861,0.196431,-0.033089
3,0,Control,BR00110363,B5,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,0.015091,0.002641,-0.028728,-0.044338,-0.000466,-0.029708,-0.010994,-0.021929,0.023909,-0.027086
4,0,Control,BR00110363,B6,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,-0.026855,-0.019703,-0.009698,-0.005962,0.034776,0.103990,0.068561,0.008069,0.051912,0.001262


In [16]:
# split the data into trianing and testing sets
meta_cols, _ = split_meta_and_features(fs_profile_df)
X = fs_profile_df[fs_feats]
y = fs_profile_df["injury_code"]

# splitting dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.80, random_state=seed, stratify=y
)

# saving training dataset as csv file
X_train.to_csv(data_split_dir / "X_train.csv.gz", compression="gzip", index=False)
X_test.to_csv(data_split_dir / "X_test.csv.gz", compression="gzip", index=False)
y_train.to_csv(data_split_dir / "y_train.csv.gz", compression="gzip", index=False)
y_test.to_csv(data_split_dir / "y_test.csv.gz", compression="gzip", index=False)

# display data split sizes
print("X training size", X_train.shape)
print("X testing size", X_test.shape)
print("y training size", y_train.shape)
print("y testing size", y_test.shape)

X training size (10802, 221)
X testing size (2701, 221)
y training size (10802,)
y testing size (2701,)


In [17]:
# save metadata after holdout
cell_injury_metadata = fs_profile_df[fs_meta]
cell_injury_metadata.to_csv(
    data_split_dir / "cell_injury_metadata_after_holdout.csv.gz",
    compression="gzip",
    index=False,
)
# display
print("Metadata shape", cell_injury_metadata.shape)
cell_injury_metadata.head()

Metadata shape (13503, 33)


,injury_code,injury_type,Plate,Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,...,Compound PubChem CID,Compound PubChem URL,Control Type,Channels,Comment [Image File Path],Comment [Image Prefix],Mahalanobis distance,Mahalanobis distance significant,Relative well cellcount,Relative well cellcount significant
0,0,Control,BR00110363,B2,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c02,7.51,No,1.02,No
1,0,Control,BR00110363,B3,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c03,6.21,No,1.11,No
2,0,Control,BR00110363,B4,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c04,10.94,No,1.02,No
3,0,Control,BR00110363,B5,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c05,7.59,No,1.06,No
4,0,Control,BR00110363,B6,Homo sapiens,NCBITaxon,NCBITaxon_9606,U2OS,EFO,EFO_0002869,...,679.0,https://pubchem.ncbi.nlm.nih.gov/compound/679,Negative,"Ch1 (blue): Nuclei, Ch2 (green): ER, Ch3 (yell...",/incoming/BR00110363/,r02c06,5.28,No,1.00,No


## Generating data split summary file 

In [18]:
def get_and_rename_injury_info(
    profile: pd.DataFrame, groupby_key: str, column_name: str
) -> pd.DataFrame:
    """Gets injury treatment information and renames the specified column.

    Parameters
    ----------
    profile : DataFrame
        The profile DataFrame containing data to be processed.
    groupby_key : str
        The key to group by in the injury treatment information.
    column_name : str
        The new name for the 'n_wells' column.

    Returns
    -------
    DataFrame
        A DataFrame with the injury treatment information and the 'n_wells' column renamed.
    """
    return get_injury_treatment_info(profile=profile, groupby_key=groupby_key).rename(
        columns={"n_wells": column_name}
    )


# name of the columns
data_col_name = [
    "Number of Wells (Total Data)",
    "Number of Wells (Train Split)",
    "Number of Wells (Test Split)",
    "Number of Wells (Plate Holdout)",
    "Number of Wells (Treatment Holdout)",
    "Number of Wells (Well Holdout)",
]


# Total amount summary
injury_before_holdout_info_df = injury_before_holdout_info_df.rename(
    columns={"n_wells": data_col_name[0]}
)

# Data splits train test summary
injury_train_info_df = get_and_rename_injury_info(
    profile=X_train.merge(
        fs_profile_df[meta_cols], how="left", right_index=True, left_index=True
    )[meta_cols + fs_feats],
    groupby_key="injury_type",
    column_name=data_col_name[1],
)

injury_test_info_df = get_and_rename_injury_info(
    profile=X_test.merge(
        fs_profile_df[meta_cols], how="left", right_index=True, left_index=True
    )[meta_cols + fs_feats],
    groupby_key="injury_type",
    column_name=data_col_name[2],
)

# Holdouts summary
injury_plate_holdout_info_df = get_and_rename_injury_info(
    profile=plate_holdout_df, groupby_key="injury_type", column_name=data_col_name[3]
)

injury_treatment_holdout_info_df = get_and_rename_injury_info(
    profile=treatment_holdout_df,
    groupby_key="injury_type",
    column_name=data_col_name[4],
)

injury_well_holdout_info_df = get_and_rename_injury_info(
    profile=wells_heldout_df, groupby_key="injury_type", column_name=data_col_name[5]
)

# Select interested columns
total_data_summary = injury_before_holdout_info_df[["injury_type", data_col_name[0]]]
train_split_summary = injury_train_info_df[["injury_type", data_col_name[1]]]
test_split_summary = injury_test_info_df[["injury_type", data_col_name[2]]]
plate_holdout_info_df = injury_plate_holdout_info_df[["injury_type", data_col_name[3]]]
treatment_holdout_summary = injury_treatment_holdout_info_df[
    ["injury_type", data_col_name[4]]
]
well_holdout_summary = injury_well_holdout_info_df[["injury_type", data_col_name[5]]]

In [19]:
# merge the summary data splits into one, update data type to integers
merged_summary_df = (
    total_data_summary.merge(train_split_summary, on="injury_type", how="outer")
    .merge(test_split_summary, on="injury_type", how="outer")
    .merge(plate_holdout_info_df, on="injury_type", how="outer")
    .merge(treatment_holdout_summary, on="injury_type", how="outer")
    .merge(well_holdout_summary, on="injury_type", how="outer")
    .fillna(0)
    .set_index("injury_type")
)[data_col_name].astype(int)

# update index and rename it 'injury_type' to "Cellular Injury"
merged_summary_df = merged_summary_df.reset_index().rename(
    columns={"injury_type": "Cellular Injury"}
)

# save as csv file
merged_summary_df.to_csv(data_split_dir / "summary_data_split.csv", index=False)

# display
merged_summary_df

,Cellular Injury,Number of Wells (Total Data),Number of Wells (Train Split),Number of Wells (Test Split),Number of Wells (Plate Holdout),Number of Wells (Treatment Holdout),Number of Wells (Well Holdout)
0,Control,9855,6726,1682,1072,0,375
1,Cytoskeletal,1472,881,221,181,12,177
2,Miscellaneous,1304,806,201,172,18,107
3,Kinase,1104,600,150,120,12,222
4,Genotoxin,944,590,147,73,48,86
5,Hsp90,552,334,84,54,0,80
6,Redox,312,172,43,54,24,19
7,Saponin,288,131,33,102,12,10
8,HDAC,168,110,28,30,0,0
9,Mitochondria,144,92,23,12,0,17
